# SQLAlchemy Playland

In [7]:
import pandas as pd
from sqlalchemy import *
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
from sqlalchemy.sql import func

from IPython.display import Image
from IPython.core.display import HTML 


chunks = pd.read_csv('./ks-projects-201801.csv')
# chunks.to_sql(name='projects', if_exists='append', con=engine)

## In this lab, we're going to be analyzing Kickstarter Data
### A place where you can get projects founded by strangers around the world. Projects both good...

In [9]:
Image(url = 'https://cms.qz.com/wp-content/uploads/2014/03/screen-shot-2014-03-25-at-7-44-00-pm.png?w=1600')

## and bad.......

In [8]:
Image(url = 'http://cdn.guff.com/site_1/media/15000/14328/items/37d0c6b4e5df2e5c04098a55.jpg')

## Let's take a look at our dataset

In [10]:
chunks.head()

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09,1000.0,2015-08-11 12:12:28,0.0,failed,0,GB,0.0,0.0,1533.95
1,1000003930,Greeting From Earth: ZGAC Arts Capsule For ET,Narrative Film,Film & Video,USD,2017-11-01,30000.0,2017-09-02 04:43:57,2421.0,failed,15,US,100.0,2421.0,30000.00
2,1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26,45000.0,2013-01-12 00:20:50,220.0,failed,3,US,220.0,220.0,45000.00
3,1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,2012-04-16,5000.0,2012-03-17 03:24:11,1.0,failed,1,US,1.0,1.0,5000.00
4,1000011046,Community Film Project: The Art of Neighborhoo...,Film & Video,Film & Video,USD,2015-08-29,19500.0,2015-07-04 08:35:03,1283.0,canceled,14,US,1283.0,1283.0,19500.00


In [2]:
Base = declarative_base()

In [3]:
class Project(Base):
    __tablename__ = "projects"
    id = Column(Integer, primary_key=True)
    name = Column(String)
    category = Column(String)
    main_category = Column(String)
    currency = Column(String)
    deadline = Column(Date)
    goal = Column(Float)
    launched = Column(Date)
    pledged = Column(Float)
    state = Column(String)
    backers = Column(Integer)
    country = Column(String)
    
engine = create_engine('sqlite:///kickstarter.db')

In [3]:
Base.metadata.create_all(engine)

In [5]:
Session = sessionmaker(bind=engine)
session = Session()

## Query Challenges

1. query the database with sqlalchemy to find the project named 'Beach Wrestling Documentary'
    - set the variable `beach_wresting_documentary` equal to the result of this query
    - return `beach_wresting_documentary.goal`
        * 20000.0
    - return `beach_wresting_documentary.backers`
        * 36
    - return `beach_wresting_documentary.state`
        * successful
        
        
2. Write a function that will return a list of Project objects who have successfully gotten funded. Check out  'successful'
    - What is the length of this list?
    
    
3. Write a function that will return a list of projects whose pledged amounts are greater than or equal to their goal amounts
    - check the length of this list of projects --> does the length of this list match the length of the list from (2)?


4. Which project has the highest number of backers?
    - **Hint** : Look into func
  
  
5. Write a function that returns a list of `main_category`s and each of their respective average `pledge` amounts.
    - **BONUS**: add in a new parameter to the function that will allow the returned list to be ordered


#### Challenge Bonus Problems


6. Write a function that will plot a histogram for the number of items found in each main_category.


7. Is there a correlation between time or day of the year and whether or not your project will be successfully backed? Investigate!


8. **Hard**: Which country has the highest percentage of successful projects?